In [1]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

from pathlib import Path

import singleton_helpers
from chia.types.blockchain_format.program import Program
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble
from clvm.casts import int_to_bytes


In [2]:
clsp = """
(mod (MOD VALUE)
    (include condition_codes.clib)
    (include utils.clib)
    (list
        (list CREATE_COIN 
            (sha256tree
                    (curry
                        MOD
                        (list
                            MOD
                            (+ VALUE 1)
                        )
                    )
            )
            1023
        )
    )
)    
"""
MOD = Program(compile_clvm_text(clsp, "."))

In [3]:
inner_puzzle = MOD.curry(MOD, int_to_bytes(0))
result = inner_puzzle.run(Program.to([]))
print(disassemble(result))

((51 0x7090f3c8e91b0c87f0e109579f0b88f1ffb74026dbdebea09090056ae7c350bb 1023))


In [4]:
next_inner_puzzle = MOD.curry(MOD, int_to_bytes(1))
print(next_inner_puzzle.get_tree_hash())

7090f3c8e91b0c87f0e109579f0b88f1ffb74026dbdebea09090056ae7c350bb


In [5]:
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

from cdv.test import Network, Wallet
from cdv.util.load_clvm import load_clvm
from clvm_tools.clvmc import compile_clvm_text

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

counter_clsp = """
(mod (MOD VALUE)
    (include condition_codes.clib)
    (include utils.clib)
    (list
        (list CREATE_COIN 
            (sha256tree
                    (curry
                        MOD
                        (list
                            MOD
                            (+ VALUE 1)
                        )
                    )
            )
            1023
        )
    )
)    
"""

counter_puzzle = Program(
    compile_clvm_text(counter_clsp, search_paths=["."])
)
curried_counter_puzzle = counter_puzzle.curry(counter_puzzle, int_to_bytes(0))

inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(curried_counter_puzzle)

standard_coin_message, coin_spends = singleton_helpers.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [("Counter", "A")]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')

# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

# push txn to create an eve singleton
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
await network.push_tx(spend_bundle)

alice: 2000000000000

starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 666d2129cd409c83df5608a1d46cba478f74bb6460f5afc40ecd7aa79620f75a>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [6]:
# spend eve singleton
network.sim.pass_blocks(1)
launcher_id = launcher_coin_spend.coin.name()
singleton_struct = (
        singleton_top_layer.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer.SINGLETON_LAUNCHER_HASH)
    )

singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )

singleton_coin = await singleton_helpers.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id) 

lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(launcher_coin_spend)

inner_solution = Program.to([])

singleton_coin_spend = singleton_helpers.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')

print('\nsingleton_coin_spend:')

from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers.print_json(spend_bundle.to_json_dict())

await network.push_tx(spend_bundle)

singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0x666d2129cd409c83df5608a1d46cba478f74bb6460f5afc40ecd7aa79620f75a'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
80

singleton_coin_spend:
{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1023,
                "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
                "puzzle_hash": "0x666d2129cd409c83df5608a1d46cba478f74bb6460f5afc40ecd7aa79620f75a"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03f

{'additions': [Coin(parent_coin_info=<bytes32: f429d7d2fdebfbe06a74a2e2f89149e36ba0ba08541c7e1a27f3782603dfb925>, puzzle_hash=<bytes32: 25eb98dd0b53888a45f83b77104106f1aba4cd1f2942af0ec7aecfe1a248bef0>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 666d2129cd409c83df5608a1d46cba478f74bb6460f5afc40ecd7aa79620f75a>, amount=1023)]}

In [7]:
curried_counter_puzzle = counter_puzzle.curry(counter_puzzle, int_to_bytes(1))
inner_puzzle = singleton_top_layer.adapt_inner_to_singleton(curried_counter_puzzle)
singleton_puzzle = singleton_top_layer.SINGLETON_MOD.curry(
            singleton_struct,
            curried_counter_puzzle,
    )
print(singleton_puzzle.get_tree_hash())

25eb98dd0b53888a45f83b77104106f1aba4cd1f2942af0ec7aecfe1a248bef0


In [8]:
# spend another singleton to increase value

lineage_proof = singleton_top_layer.lineage_proof_for_coinsol(singleton_coin_spend)

singleton_coin = await singleton_helpers.get_unspent_singleton(get_coin_records_by_parent_ids, launcher_id)
inner_solution = Program.to([])


singleton_coin_spend = singleton_helpers.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')

print('\nsingleton_coin_spend:')
singleton_helpers.print_json(singleton_coin_spend.to_json_dict())
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
await network.push_tx(spend_bundle)

singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0xf429d7d2fdebfbe06a74a2e2f89149e36ba0ba08541c7e1a27f3782603dfb925',
 'puzzle_hash': '0x25eb98dd0b53888a45f83b77104106f1aba4cd1f2942af0ec7aecfe1a248bef0'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': '0xe36327e14904c9e56d916767b2fa4a7e6f01e1f67d5fa19f9434355211dd07bd',
 'parent_name': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570'}

inner_solution:
80

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0xf429d7d2fdebfbe06a74a2e2f89149e36ba0ba08541c7e1a27f3782603dfb925",
        "puzzle_hash": "0x25eb98dd0b53888a45f83b77104106f1aba4cd1f2942af0ec7aecfe1a248bef0"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fffff010180ffff01ff02ff36ffff04ff02ffff04ff05ffff04ff17ffff04ffff02ff26ffff04ff02ffff04ff0bff80808080ffff04ff2fffff04ff0bffff04ff5fff808080808080808080ffff01ff088080ff0180ffff04ffff01ffffffff4602ff3304ffff0101ff02ffff02ffff03ff05ffff01f

{'additions': [Coin(parent_coin_info=<bytes32: 4349baa74193e80f59c6a1cc476a2f26dd187951259c1d25f87f0f142adc613c>, puzzle_hash=<bytes32: 35694c9f8eb8e67f75e95e29d25ff6a2ef710d013b612d16438cc8530eef2512>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: f429d7d2fdebfbe06a74a2e2f89149e36ba0ba08541c7e1a27f3782603dfb925>, puzzle_hash=<bytes32: 25eb98dd0b53888a45f83b77104106f1aba4cd1f2942af0ec7aecfe1a248bef0>, amount=1023)]}